In [1]:
import sys
sys.path.append('../xrun')

In [27]:
import re

from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")

from IPython.core.display import HTML
from sklearn.metrics import pairwise_distances

from xrun.data.run_info import RunInfo
from xrun.data.loader import load_dataset

In [46]:
data_results_dir = Path("../data/odin-results/hardinstanceb1/")
run_files = list(data_results_dir.glob("**/*.json"))
run_files = list(Path("../data/cary-results/hardinstanceb1").glob("**/*.json"))

In [47]:
run_files

[PosixPath('../data/cary-results/hardinstanceb1/group-sampling-k20-m4000/2021-08-07-09-40-10/000-hardinstanceb1-group-sampling-k20-m4000.json'),
 PosixPath('../data/cary-results/hardinstanceb1/group-sampling-k20-m4000/2021-08-09-09-10-26/011-hardinstanceb1-group-sampling-k20-m4000.json'),
 PosixPath('../data/cary-results/hardinstanceb1/group-sampling-k20-m4000/2021-08-10-16-40-01/019-hardinstanceb1-group-sampling-k20-m4000.json'),
 PosixPath('../data/cary-results/hardinstanceb1/group-sampling-k20-m4000/2021-08-10-08-55-35/017-hardinstanceb1-group-sampling-k20-m4000.json'),
 PosixPath('../data/cary-results/hardinstanceb1/group-sampling-k20-m4000/2021-08-09-01-18-11/009-hardinstanceb1-group-sampling-k20-m4000.json'),
 PosixPath('../data/cary-results/hardinstanceb1/group-sampling-k20-m4000/2021-08-09-21-55-19/014-hardinstanceb1-group-sampling-k20-m4000.json'),
 PosixPath('../data/cary-results/hardinstanceb1/group-sampling-k20-m4000/2021-08-08-21-03-03/008-hardinstanceb1-group-sampling-k20

In [41]:
run_info_path = run_files[6]

In [42]:
coreset_file_path = run_info_path.parent / "results.txt.gz"
centers_file_path = run_info_path.parent / "centers.txt"

In [43]:
run_info = RunInfo.load_json(run_info_path)

In [44]:
run_info

RunInfo(algorithm='basic-clustering', dataset='hardinstanceb1', k=20, m=4000, iteration=6, randomSeed=7145266, output_dir='data/experiments/hardinstanceb1/basic-clustering-k20-m4000/2021-08-14-21-10-14', command='gs/build/gs basic-clustering hardinstanceb1 data/input/benchmark-k20-alpha5-beta1.00.txt.gz 20 4000 7145266 data/experiments/hardinstanceb1/basic-clustering-k20-m4000/2021-08-14-21-10-14', start_time='2021-08-14T21:10:14.893143', end_time='2021-08-15T14:08:27.279512', duration_secs=61092.386369, process_id=-2)

In [30]:
original_data = load_dataset(f"../{run_info.dataset_path}")

Loading csv dataset from ../data/input/benchmark-k20-alpha5-beta1.00.txt.gz...
Loaded matrix of shape (3200000, 100) in 55.86 secs


In [31]:
original_data

array([[ 0.95, -0.05, -0.05, ..., -0.05, -0.05, -0.05],
       [ 0.95, -0.05, -0.05, ..., -0.05, -0.05, -0.05],
       [ 0.95, -0.05, -0.05, ..., -0.05, -0.05, -0.05],
       ...,
       [-0.05, -0.05, -0.05, ...,  0.95, -0.05, -0.05],
       [-0.05, -0.05, -0.05, ..., -0.05,  0.95, -0.05],
       [-0.05, -0.05, -0.05, ..., -0.05, -0.05,  0.95]])

In [40]:
coreset_file_path

PosixPath('../data/odin-results/hardinstanceb1/basic-clustering-k20-m4000/2021-08-17-02-09-46/results.txt.gz')

In [16]:
coreset = np.loadtxt(fname=coreset_file_path, dtype=np.double, delimiter=' ', skiprows=1)
coreset_weights = coreset[:,0]
coreset_points = coreset[:,1:]

In [17]:
centers = np.loadtxt(fname=centers_file_path, dtype=np.double, delimiter=' ', skiprows=0)
center_weights = centers[:,0] 
center_points = centers[:,1:]

In [20]:
coreset_points.shape

(4000, 100)

In [38]:
coreset_points[100]

array([ 0.539377 , -0.0254852, -0.0237828, -0.0254852, -0.0319544,
       -0.0275281, -0.0265066, -0.0275281, -0.0316139, -0.038764 ,
       -0.0309329, -0.0282091, -0.0258257, -0.0285495, -0.0265066,
       -0.0309329, -0.0285495, -0.0265066, -0.029571 , -0.0251447,
       -0.029571 , -0.0271876, -0.0285495,  0.553337 , -0.038764 ,
       -0.0309329, -0.0285495, -0.0258257, -0.0278686, -0.0268471,
       -0.0265066, -0.0391045, -0.0265066, -0.0282091, -0.0299115,
       -0.0329758, -0.0261662, -0.0251447, -0.0299115, -0.0248042,
       -0.0285495, -0.0329758,  0.598621 , -0.0278686, -0.02889  ,
       -0.0282091, -0.0309329, -0.0329758, -0.0278686, -0.0312734,
       -0.02889  , -0.0312734, -0.0408069, -0.0333163, -0.0275281,
       -0.038764 , -0.0309329, -0.030252 , -0.0282091, -0.0391045,
       -0.0244637, -0.0278686, -0.0278686, -0.0363807, -0.0282091,
       -0.0271876, -0.0316139, -0.0282091, -0.0265066, -0.0305924,
       -0.02889  , -0.0261662, -0.0217399, -0.0275281, -0.0248

In [39]:
coreset_points[101]

array([-0.023919 , -0.0287234, -0.0273507, -0.0266644,  0.540597 ,
       -0.0290666, -0.0376458, -0.040048 , -0.0304393, -0.0297529,
       -0.025978 , -0.0242622, -0.025978 , -0.0273507, -0.0290666,
       -0.0276939, -0.0270075, -0.0270075, -0.0276939, -0.0249485,
       -0.0263212, -0.025978 , -0.0249485, -0.0376458, -0.0321551,
        0.5128   , -0.0266644, -0.0280371, -0.0270075, -0.0266644,
       -0.0273507, -0.025978 , -0.0225463, -0.0266644, -0.025978 ,
       -0.0242622, -0.0252917, -0.0266644, -0.025978 , -0.0266644,
       -0.0242622, -0.0270075, -0.0283802, -0.0273507, -0.0276939,
        0.572512 , -0.0283802, -0.0352437, -0.0287234, -0.0270075,
       -0.0386754, -0.0270075, -0.0441661, -0.0273507, -0.0273507,
       -0.0283802, -0.0321551, -0.0283802, -0.0373027, -0.0276939,
       -0.0294097, -0.0246054, -0.0390185, -0.0352437, -0.0321551,
       -0.0256349, -0.0318119, -0.0280371, -0.025978 , -0.0342141,
       -0.0276939, -0.0252917, -0.0383322, -0.0266644, -0.0300